In [30]:
import os
import random as rn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import plot_model
from IPython.display import Image

In [31]:
import os
root_dir = "C:\\Users\\saiki\\OneDrive\\Desktop\\BTech\\data sets\\Rice_Image_Dataset\\Rice_Image_Dataset"
f_names = [f_name for f_name in os.listdir(root_dir)]
print(f"Folder (or file) names:\n{f_names}")

Folder (or file) names:
['Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag']


In [32]:
rice_classes = f_names.copy()
print(f"Rice classes:\n{', '.join(rice_classes)}")

Rice classes:
Arborio, Basmati, Ipsala, Jasmine, Karacadag


In [33]:
classes_counts = {rice_class: len(os.listdir(os.path.join(root_dir, rice_class))) for rice_class in rice_classes}

In [34]:
rice_list = []
for rice_class in rice_classes:
    class_dir = os.path.join(root_dir, rice_class)
    for img_path in os.listdir(class_dir):
        full_path = os.path.join(class_dir, img_path)
        rice_list.append((full_path, rice_class))

In [35]:
df = pd.DataFrame(rice_list, columns=["File_Path", "Label"])

In [36]:
df = df.sample(frac=1).reset_index(drop=True)

In [37]:
total_samples = df.shape[0]
print(f"Total number of samples is {total_samples}")

Total number of samples is 75000


In [38]:
0.15 * total_samples, 0.7 * total_samples

(11250.0, 52500.0)

In [39]:
train_set = df[:52500].copy() # 70% of data
validate_set = df[52500:63750].copy()  # 15% of data
test_set = df[63750:].copy() # 15% of data

In [40]:
epochs = 10
batch_size = 128
img_height = 224
img_width = 224

In [41]:
data_generator = ImageDataGenerator(rescale=1./255)
train_generator = data_generator.flow_from_dataframe(
  dataframe=train_set,
  x_col="File_Path",
  y_col="Label",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode="categorical",
  shuffle=True,
  seed=42
)

Found 52500 validated image filenames belonging to 5 classes.


In [42]:
validation_generator = data_generator.flow_from_dataframe(
  dataframe=validate_set,
  x_col="File_Path",
  y_col="Label",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

Found 11250 validated image filenames belonging to 5 classes.


In [43]:
test_generator = data_generator.flow_from_dataframe(
  dataframe=test_set,
  x_col="File_Path",
  y_col="Label",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

Found 11250 validated image filenames belonging to 5 classes.


In [44]:
for image_batch, labels_batch in train_generator:
    print(f"Train Shape: {image_batch.shape} (Batches = {len(train_generator)})")
    print(f"Train label: {labels_batch.shape}\n")
    break
    
for image_batch, labels_batch in validation_generator:
    print(f"Validation Shape: {image_batch.shape} (Batches = {len(validation_generator)})")
    print(f"Validation label: {labels_batch.shape}\n")
    break
    
for image_batch, labels_batch in test_generator:
    print(f"Test Shape: {image_batch.shape} (Batches = {len(test_generator)})")
    print(f"Test label: {labels_batch.shape}\n")
    break

Train Shape: (128, 224, 224, 3) (Batches = 411)
Train label: (128, 5)

Validation Shape: (128, 224, 224, 3) (Batches = 88)
Validation label: (128, 5)

Test Shape: (128, 224, 224, 3) (Batches = 88)
Test label: (128, 5)



In [45]:
AlexNet_model = models.Sequential([
    layers.Conv2D(96, 11, strides=4, activation='relu', input_shape=(224, 224, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Conv2D(256, 5, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Conv2D(384, 3, activation='relu'),
    layers.Conv2D(384, 3, activation='relu'),
    layers.Conv2D(256, 3, activation='relu'),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Flatten(),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4096, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='softmax')
])

C:\Users\saiki\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
AlexNet_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 54, 54, 96)          │          34,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 54, 54, 96)          │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 96)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 22, 22, 256)         │         614,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 22, 22, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 10, 10, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 8, 8, 384)           │         885,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 6, 6, 384)           │       1,327,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 4, 4, 256)           │         884,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4096)                │       1,052,672 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 4096)                │      16,781,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │          20,485 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,603,077 (82.41 MB)

 Trainable params: 21,602,373 (82.41 MB)

 Non-trainable params: 704 (2.75 KB)

In [47]:
AlexNet_model.compile(loss=BinaryCrossentropy(),
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['accuracy'])

In [19]:
AlexNet = AlexNet_model.fit(train_generator,
                    epochs=epochs,
                    validation_data=validation_generator)

C:\Users\saiki\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2392s 6s/step - accuracy: 0.8027 - loss: 0.2806 - val_accuracy: 0.6644 - val_loss: 0.2766
Epoch 2/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 1191s 3s/step - accuracy: 0.9636 - loss: 0.0449 - val_accuracy: 0.6666 - val_loss: 0.7491
Epoch 3/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 1045s 3s/step - accuracy: 0.9770 - loss: 0.0293 - val_accuracy: 0.5585 - val_loss: 0.5306
Epoch 4/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 1045s 3s/step - accuracy: 0.9823 - loss: 0.0237 - val_accuracy: 0.7820 - val_loss: 0.2519
Epoch 5/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2256s 5s/step - accuracy: 0.9810 - loss: 0.0243 - val_accuracy: 0.9438 - val_loss: 0.0546
Epoch 6/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2166s 5s/step - accuracy: 0.9863 - loss: 0.0200 - val_accuracy: 0.9183 - val_loss: 0.0987
Epoch 7/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 3202s 8s/step - accuracy: 0.9881 - loss: 0.0167 - val_accuracy: 0.6334 - val_loss: 0.8222
Epoch 8/10
411/411 ━━━━━━━━━━━━━━━━━━━━ 2013s 5s/step - accuracy: 0.9870 - loss: 0.0166 - 

In [22]:
AlexNet_model.save('AlexNet_model.keras')
history_alex = AlexNet.history

In [48]:
load_Alex = tf.keras.models.load_model('AlexNet_model.keras')

C:\Users\saiki\AppData\Roaming\Python\Python310\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 22 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [49]:
def plot_confusion_matrix(conf_matrix, class_labels, model_name):
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix of {model_name}')
    plt.show()

In [50]:
def print_classification_report(true_classes, predicted_classes, class_labels, model_name):
    print(f"Classification Report for {model_name} Model:\n")
    print(classification_report(true_classes, predicted_classes, target_names=class_labels))

In [51]:
def evaluate_model(load_model, test_generator, class_labels, model_name):
    predictions = load_model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes
    conf_matrix = confusion_matrix(true_classes, predicted_classes)   
    return conf_matrix, true_classes, predicted_classes

In [52]:
conf_matrix_AlexNet, true_classes_AlexNet, predicted_classes_AlexNet = evaluate_model(load_Alex, test_generator, rice_classes, "AlexNet")

C:\Users\saiki\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


88/88 ━━━━━━━━━━━━━━━━━━━━ 85s 954ms/step


In [53]:
np.sum(predictions != labels)

495

In [52]:
np.sum(predictions == labels)

145

In [49]:
def plot_confusion_matrix(conf_matrix, class_labels, model_name):
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Greens', xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title(f'Confusion Matrix of {model_name}')
    plt.show()

def print_classification_report(true_classes, predicted_classes, class_labels, model_name):
    print(f"Classification Report for {model_name} Model:\n")
    print(classification_report(true_classes, predicted_classes, target_names=class_labels))

def evaluate_model(load_model, test_generator, class_labels, model_name):
    predictions = load_model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes
    conf_matrix = confusion_matrix(true_classes, predicted_classes)
    
    return conf_matrix, true_classes, predicted_classes
conf_matrix_AlexNet, true_classes_AlexNet, predicted_classes_AlexNet = evaluate_model(load_Alex, test_generator, rice_classes, "AlexNet")

88/88 ━━━━━━━━━━━━━━━━━━━━ 152s 2s/step


In [24]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
model_path = "AlexNet_model.keras" 
model = tf.keras.models.load_model(model_path)

img_path = "C:\\Users\\saiki\\OneDrive\\Desktop\\BTech\\data sets\\Rice_Image_Dataset\\Rice_Image_Dataset\\Basmati\\Basmati (1).jpg"

print("Model expects input shape:", model.input_shape) 

expected_size = (227, 227) if model.input_shape[1] == 227 else (224, 224)
img = image.load_img(img_path, target_size=expected_size)  

img_array = image.img_to_array(img)  
img_array = np.expand_dims(img_array, axis=0)  
img_array = img_array.astype("float32") / 255.0  

print("Image shape before prediction:", img_array.shape)  

predictions = model.predict(img_array)
print("Predictions:", predictions)


Model expects input shape: (None, 224, 224, 3)
Image shape before prediction: (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 569ms/step
Predictions: [[4.3225264e-06 9.8220336e-01 1.0033370e-05 1.7780585e-02 1.6814392e-06]]


In [26]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

rice_classes = ['basmati', 'arborio', 'ispala', 'karacadag', 'jasmine']

model_path = "AlexNet_model.keras"  
model = tf.keras.models.load_model(model_path)
img_path = "C:\\Users\\saiki\\OneDrive\\Desktop\\BTech\\data sets\\Rice_Image_Dataset\\Rice_Image_Dataset\\Basmati\\Basmati (1).jpg"
input_size = model.input_shape[1:3] 
print("Model expects input size:", input_size)
img = image.load_img(img_path, target_size=input_size)
img_array = image.img_to_array(img) 
img_array = np.expand_dims(img_array, axis=0) 
img_array = img_array.astype("float32") / 255.0  

print("Image shape before prediction:", img_array.shape)

predictions = model.predict(img_array)

predicted_class_index = np.argmax(predictions)  
predicted_class = rice_classes[predicted_class_index]  

print("Predictions:", predictions)
print(f"Predicted Rice Class: {predicted_class} (Index: {predicted_class_index})")


Model expects input size: (224, 224)
Image shape before prediction: (1, 224, 224, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step
Predictions: [[4.3225264e-06 9.8220336e-01 1.0033370e-05 1.7780585e-02 1.6814392e-06]]
Predicted Rice Class: arborio (Index: 1)
